# Web app

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import base64
import io
import numpy as np
import tensorflow as tf
from PIL import Image
import os

# Charger le modèle de deep learning pré-entraîné
model_path = "model.keras"  # Remplacez par le chemin vers votre modèle
model = tf.keras.models.load_model(model_path)

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Définir la mise en page de l'application
app.layout = html.Div([
    html.H1("Prediction app", style = {'text-align': 'center'}),
    dcc.Upload(
        id="upload-image",
        children=html.Button("Upload an image", style = {'width': '150px', 'margin': '5px', 'padding': '8px',
                           'background-color': 'rgb(93, 93, 208)', 'border-radius': '10px'}),
        multiple=False,
        style = {'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}
    ),
    html.Div(id="output-image", style = {'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
    
    # Définir la mise en page de l'application
    html.Div(
            children=[
                html.Button('Predict', id='predict-button', style = {'width': '150px', 'margin': '5px', 'padding': '8px',
                           'background-color': 'rgb(22, 171, 22)', 'border-radius': '10px'})
                ], style = {'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
    
    # Définir la mise en page de l'application
   # html.Button("Prédire", id="predict-button"),
    html.Div(id="prediction-output", style = {'display': 'flex', 'justify-content': 'center', 'align-items': 'center'})
],
  style = {
        'border': '1px solid black',
        'width': '600px',
        'position': 'relative',
        'position': 'absolute',
        'top': '50%', 
        'left': '50%',
        'transform': 'translate(-50%,-50%)',
        'padding-left': '10px',
        'padding-right': '10px',
        'background-color': 'antiquewhite'
    }

)

# Stocker le chemin absolu de l'image téléchargée
uploaded_image_path = None

# Callback pour gérer le téléchargement d'images
@app.callback(
    Output("output-image", "children"),
    Input("upload-image", "contents"),
    State("upload-image", "filename")
)
def update_image(contents, filename):
    global uploaded_image_path
    if contents is not None:
        image = base64.b64decode(contents.split(",")[1])
        img = Image.open(io.BytesIO(image))
        img = img.resize((120, 200))  # Redimensionner l'image
        uploaded_image_path = os.path.abspath(filename)  # Stocker le chemin absolu
        return html.Img(src=contents, style={"width": "30%"})

# Callback pour gérer la prédiction
@app.callback(
    Output("prediction-output", "children"),
    Input("predict-button", "n_clicks"),
)
def predict_image(n_clicks):
    global uploaded_image_path
    if uploaded_image_path is not None:
        # Define image dimensions and batch size
        img_height, img_width = 180, 180
        img = tf.keras.preprocessing.image.load_img(uploaded_image_path, target_size=(img_height, img_width))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create a batch
        # Interpret the predictions
        predictions = model.predict(img_array)
        class_names = ["Adult", "Child"]
        predicted_class_index = np.argmax(predictions)
        predicted_class = class_names[predicted_class_index]
        return html.H3(f"Prediction : {predicted_class}")

if __name__ == "__main__":
    app.run_server(debug=True)

1/1 [==============================] - 0s 213ms/step
